# **AVISO EN CASO DE ERRORES**

En caso de que la librería de pandas-profiling de errores de ejecución (lo cual nos ha resultado bastante común), ejecute estas líneas de código y reinicie el entorno de ejecución para que funcione la parte del profiling correctamente en caso de que se quiera ejecutar).

In [ ]:
%%capture
!pip uninstall -y pandas-profiling
!pip install pandas-profiling

# Descripción del Colab

En este Colab se muestran los procesos de limpieza de datos y profiling de los distintos datasets que hemos empleado para el laboratorio de Minería de Datos. 

La explicación de los procesos realizados para cada dataset se muestran separados a continuacion en este Colab.

### Librerías empleadas en este fichero

In [25]:
%%capture
!pip install deep_translator

In [26]:
import numpy as np
import pandas as pd
from pandas_profiling import ProfileReport
from pandas.io.formats.style_render import DataFrame
from google.colab import files
import io
from deep_translator import GoogleTranslator
import sys

# Carga de datos
Ejecución de código para seleccionar los archivos que les hemos otorgado paa subirlos en el Colab.

In [ ]:
uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

# Dataset 1 - Ventas y precios nacionales
Ahora vamos a cargar los datos en el dataframe llamado "dataframe_1".

In [129]:
dataframe_1 = pd.read_csv("Dataset1.- DatosConsumoAlimentarioMAPAporCCAA.txt", sep="|")

## Limpieza de columnas

Aquí vamos a eliminar las columnas que no nos aportan nada para nuestro objetivo, que en este caso sería ver el precio de unos productos y el consumo de estos en España. En este caso sólo vamos a querer las columnas "Año", "Mes", "CCAA" (esta la eliminaremos después porque la necesitamos para algo antes), "Producto", "Precio medio kg" y "Consumo per capita".

In [119]:
del dataframe_1["Penetración (%)"]
del dataframe_1["Valor (miles de €)"]
del dataframe_1["Gasto per capita"]
del dataframe_1["Volumen (miles de kg)"]
del dataframe_1["Unnamed: 10"]
del dataframe_1["Unnamed: 11"]

Aquí cambiamos el nombre de las columnas "Precio medio kg" y "Consumo per capita" para reemplazarle los espacios del nombre por "_". Esto es para manipular los datos de forma más comoda en adelante.

In [120]:
dataframe_1.rename({'Precio medio kg': 'Precio_medio_kg'}, axis=1, inplace=True)
dataframe_1.rename({'Consumo per capita': 'Consumo_per_capita'}, axis=1, inplace=True)

dataframe_1.head()

,Año,Mes,CCAA,Producto,Precio_medio_kg,Consumo_per_capita
0,2018,Enero,Total Nacional,TOTAL PATATAS,"0,78","2,38"
1,2018,Enero,Total Nacional,PATATAS FRESCAS,"0,69","1,74"
2,2018,Enero,Total Nacional,PATATAS CONGELADAS,"1,21","0,09"
3,2018,Enero,Total Nacional,PATATAS PROCESADAS,"5,02","0,11"
4,2018,Enero,Total Nacional,T.HORTALIZAS FRESCAS,"1,79","4,6"


A continuación se ejecuta la siguiente línea de código para saber si hay elementos nulos (vemos que no los hay).

In [121]:
dataframe_1.isnull().values.all()

False

### Limpieza de filas
Primero vamos a quitar los datos de las comunidades autónomas dejando solo los datos nacionales. Esto es debido a que los datos de comunidades autónomas los tenemos en otro dataset ya más completo.
A continuación eliminamos la columna de CCAA, como dijimos previamente, porque ya no la necesitamos.

In [122]:
df1_removed_rows = dataframe_1.drop(dataframe_1[dataframe_1.CCAA != "Total Nacional"].index)
del df1_removed_rows["CCAA"]

A continuación primero se cambia el nombre del producto "TOTAL PATATAS" a "PATATAS" debido a que hay muchos tipos de patatas y este los aúna todos. Entonces una vez cambiado el nombre eliminamos los demas productos referentes a patatas específicas.
Finalmente eliminamos los productos que aúnan multiples productos en uno solo estilo "Total hortalizas frescas" que lo único que hace es sumar los datos de todas las hortalizas.

In [123]:
df1_removed_rows['Producto'].replace({'TOTAL PATATAS': 'PATATAS'}, inplace=True)
df1_removed_rows.drop(df1_removed_rows[df1_removed_rows.Producto == "PATATAS FRESCAS"].index, inplace=True)
df1_removed_rows.drop(df1_removed_rows[df1_removed_rows.Producto == "PATATAS CONGELADAS"].index, inplace=True)
df1_removed_rows.drop(df1_removed_rows[df1_removed_rows.Producto == "PATATAS PROCESADAS"].index, inplace=True)
df1_removed_rows.drop(df1_removed_rows[df1_removed_rows.Producto == "T.HORTALIZAS FRESCAS"].index, inplace=True)
df1_removed_rows.drop(df1_removed_rows[df1_removed_rows.Producto == "OTR.HORTALIZAS/VERD."].index, inplace=True)
df1_removed_rows.drop(df1_removed_rows[df1_removed_rows.Producto == "VERD./HORT. IV GAMA"].index, inplace=True)
df1_removed_rows.drop(df1_removed_rows[df1_removed_rows.Producto == "T.FRUTAS FRESCAS"].index, inplace=True)
df1_removed_rows.drop(df1_removed_rows[df1_removed_rows.Producto == "OTRAS FRUTAS FRESCAS"].index, inplace=True)
df1_removed_rows.drop(df1_removed_rows[df1_removed_rows.Producto == "FRUTAS IV GAMA"].index, inplace=True)

Tras eliminar todas estas filas problemáticas, se decide reindexar el dataset

In [124]:
df1_removed_rows = df1_removed_rows.reset_index(drop=True)

Aquí se comprueba qué tipo es cada columna por si hay que cambiar alguna.

In [125]:
df1_removed_rows.dtypes

Año                    int64
Mes                   object
Producto              object
Precio_medio_kg       object
Consumo_per_capita    object
dtype: object

Finalmente se decide que si y se cambia el tipo de la columna "Mes" a string, el de la columna "Producto" a string, el de la columna "Precio_medio_kg" a float y el de la columna "Consumo_per_capita" a float.

In [126]:
df1_removed_rows['Mes'] = df1_removed_rows['Mes'].astype('string')
df1_removed_rows['Producto'] = df1_removed_rows['Producto'].astype('string')

df1_removed_rows['Precio_medio_kg'] = df1_removed_rows['Precio_medio_kg'].str.replace(',', '.').astype(float)
df1_removed_rows['Consumo_per_capita'] = df1_removed_rows['Consumo_per_capita'].str.replace(',', '.').astype(float)

df1_removed_rows.dtypes

Año                     int64
Mes                    string
Producto               string
Precio_medio_kg       float64
Consumo_per_capita    float64
dtype: object

### Descarga del archivo limpiado
Se tranforma el dataframe a csv separando las celdas por el símbolo ";", que es el símbolo que lee excel por defecto como separador.

In [ ]:
df1_removed_rows.to_csv('df_Spain_data.csv', sep = ';', index = False)

files.download('df_Spain_data.csv')

## Profiling del dataset 1

Hacemos profiling del primer dataset para poder analizar mejor los datos contenidos en el dataset final una vez lo hemos limpiado.

In [ ]:
profile = ProfileReport(df1_removed_rows, title="Consumo alimentario en España", html={'style':{'full_width': True}})
profile.to_notebook_iframe()

# Dataset 2 - Andalucía y Comunidad de Madrid

Este dataset está centrado en los precios, volumen y valores de ventas mensuales de ciertos productos agrícolas en las comunidades de Andalucía y de Madrid.

La principal intención de este dataset es ser empleado para la hipótesis 2: 

*Las zonas con mayor densidad de población tienen un mayor número de ventas de productos agrícolas.*

### Descripción de valores y columnas en el dataset
Dentro de este dataset nos encontramos un total de 8 columnas:

*   **Producto**: lista de los productos contenidos en el dataset.
*   **Fecha**: muestra el mes y el año de los datos.

En las siguientes columnas nos hemos ahorrado en explicarlas por separado, pues son 6 columnas que se diferencian en mostrar la misma información cambiando de qué comunidad (COM) provienen los datos. Por lo que se pueden resumir en 3:
*   **Precio COM (€/Kg)**: Esta columna muestra el valor del precio medio de un producto en la fecha indicada para una región.
*   **Valor COM (miles €)**: Esta columna muestra el valor total de las ganancias de las ventas de un producto en la fecha indicada para una región.
*   **Volumen COM (miles kg)**: Esta columna muestra el volumen total de las ventas de un producto en la fecha indicada para una región.

### Proceso de limpieza de los datos

En este apartado explicaremos qué procesos se han seguido para limpiar los datos, de dónde provienen estos y qué información contenían originalmente.

Los datos originales de los datos provienen en 3 archivos excel, cuya información viene dividida por meses en hojas de excel. Para poder leer correctamente los datos de los archivos se crea un bucle for que itere entre los archivos con otro bucle anidado que itere por las hojas dentro de estos.

Para poder seleccionar los datos que nos interesan guardamos en la primera iteración del bucle la lista de los productos que nos interesen en la lista llamada ***products***. Esta lista nos sirve para poder hacer un *merge* entre la lista *products* y la hoja con todos los datos que vamos leyendo. De esta manera nos quedamos en la variable ***mes_aux*** los productos con su información relevante del mes que estemos leyendo.

Tras filtrar los datos del mes en cuestión, eliminamos los duplicados que haya. Esto se debe a que los datos originales muestra información sobre distintos tipos de tomates, pimientos o judías verdes, entre otros, con la única distinción visual de incluírlos en subapartados (como productos frescos, de conserva o congelados) que no podemos diferenciar a través de código. Por esto, se decide eliminar los duplicados, manteniendo únicamente la primera aparición de los datos, ya que sabemos que son los datos de las hortalizas y frutas frescas, no de conserva ni congelados.

In [ ]:
# Declaración de variables
files =['2018datosmensualesdelpaneldeconsumoalimentarioenhogares_tcm30-520451_tcm30-520451.xlsx',
        '2019datosmensualesdelpaneldeconsumoalimentarioenhogares_tcm30-5204501_tcm30-520450.xlsx',
        '2020-datos-mensuales-panel-hogares-ccaa-rev-nov2021_tcm30-540244.xlsx']
lista_mes = []
lista_mes_2020 = []
meses = {'Enero':'01', 'Febrero':'02', 'Marzo':'03', 'Abril':'04', 'Mayo':'05', 'Junio':'06', 'Julio':'07',
         'Agosto':'08', 'Septiembre':'09', 'Octubre':'10', 'Noviembre':'11', 'Diciembre':'12'}
first = True
products = pd.DataFrame(columns = ['CONSUMO EN HOGARES'])
mes_aux = pd.DataFrame(columns = ['Producto', 'Fecha', 'Precio AND (€/Kg)', 'Precio MAD (€/Kg)', 'Valor AND (miles €)', 
                            'Valor MAD (miles €)', 'Volumen AND (miles kg)', 'Volumen MAD (miles kg)'])

# Bucle para la limpieza y obtención de los datos de los datasets originales
for file in files:
  año = file[0:4]
  condition = True
  
  # Diferenciamos entre 2018/19 y 2020 al cambiar de formato para nombrar las hojas del excel
  if año == '2020':
    meses = {k.lower(): v for k, v in meses.items()}
    condition = False

  for mes in meses:
    xl = pd.read_excel(file, sheet_name = mes)
    mes_2020 = mes_aux = pd.DataFrame(columns = ['Producto', 'Fecha', 'Precio AND (€/Kg)', 
                                                 'Precio MAD (€/Kg)', 'Valor AND (miles €)', 
                                                 'Valor MAD (miles €)', 'Volumen AND (miles kg)', 
                                                 'Volumen MAD (miles kg)'])
    # Take list of products we are interested in
    if mes == 'Enero' and first:
      products['CONSUMO EN HOGARES'] = xl.iloc[420:465]['CONSUMO EN HOGARES']
      first = False
    
    # Juntamos el dataset base con la lista de productos con un merge para obtener solo los datos de interés
    first_month = pd.merge(products, xl, on='CONSUMO EN HOGARES', how='left')
    mes_aux[['Producto', 'Precio AND (€/Kg)', 'Precio MAD (€/Kg)', 'Valor AND (miles €)', 'Valor MAD (miles €)', 
             'Volumen AND (miles kg)', 'Volumen MAD (miles kg)']] = first_month[['CONSUMO EN HOGARES', 
             'Unnamed: 40', 'Unnamed: 46', 'Unnamed: 41', 'Unnamed: 47', 'Unnamed: 42', 'Unnamed: 48']]
    
    mes_aux['Fecha'] = meses[mes]+'/'+año
    
    # Limpieza de duplicados
    mes_aux = mes_aux.drop_duplicates(subset=['Producto'], keep='first')

    # Unimos los datos de cada mes y los almacenamos en un dataset aparte
    lista_mes.append(mes_aux)
    dataset_andmad = pd.concat(lista_mes)

# Limpieza de nulos y normalización de los datos
dataset_andmad = dataset_andmad.convert_dtypes()
dataset_andmad = dataset_andmad.fillna(0.0)

# Nos quitamos los valores que no nos interesan
not_wanted = ['T.FRUTAS FRESCAS', 'T.HORTALIZAS FRESCAS', 'OTRAS FRUTAS FRESCAS', 
              'OTR.HORTALIZAS/VERD.', 'VERD./HORT. IV GAMA']
ds_clean = dataset_andmad

for product in not_wanted:
  ds_clean = ds_clean.drop(ds_clean[ds_clean['Producto'] == product].index)

ds_clean = ds_clean.round(decimals=3)
ds_clean.reset_index(drop=True)

## Profiling del dataset 2

Hacemos profiling del segundo dataset para poder analizar mejor los datos contenidos en el dataset final una vez lo hemos limpiado.

In [ ]:
# Profiling of our dataset
profile = ProfileReport(ds_clean, title="Andalucia-Madrid", html={'style':{'full_width': True}})
profile.to_notebook_iframe()

# Dataset 4

In [37]:
df = pd.read_csv("Dataset4.- Comercio Exterior de España.txt", sep ="|")
df['FLOW'] = df['FLOW'].astype('category')
df.Value= df.Value.replace({":":0})

translator = GoogleTranslator(source="en", target="es")

counts1 = df.groupby("INDICATORS").size()
df1 = df[(df.INDICATORS == "VALUE_IN_EUROS")]
df2 = df[(df.INDICATORS == "QUANTITY_IN_100KG")]
df2.index=df1.index
df1.insert(7, "VALUE IN EUROS", df1.Value, True)
df1.insert(8, "QUANTITY IN 100 KG", df2.Value, True)

del(df1['INDICATORS'])
del(df1['Value'])

df1.isnull().values.any()

False

In [ ]:
counts = df1.groupby("PRODUCT").size()
for i in counts.index:
  df1.PRODUCT= df1.PRODUCT.replace({i:translator.translate(i)})
df1 = df1.drop(columns=['PARTNER'])

df1.to_csv(r'Dataset 4 actualizado', index=False)

## Profiling del dataset 4

Hacemos profiling del *cuarto* dataset para poder analizar mejor los datos contenidos en el dataset final una vez lo hemos limpiado.

In [ ]:
profile = ProfileReport(df1, title="Imports-Exports", html={'style':{'full_width': True}})
profile.to_notebook_iframe()

# Descripción: Dataset 5
Descartadas columnas: 
- 'dateRep': redundundante, ya se utilizan 'day', 'month' y 'year' para describir la fecha, en un formato mas util para la selección con respecto al estudio a realizar.
- 'geoId': irrelevante, es mas descriptivo el uso de 'countriesAndTerritories' para vilualizar los datos por país.
- 'countryterritoryCode': al igual de 'geoId' es una abreviatura del país al que hace referencia.
- 'popData2019': irrelevante, para el estudio los valores de la poblacion durante 2019 son innecesario. No se va a comparar la muestra de 2019 con muestras de los años siguiestes.

Descartadas filas cuyo indice coincide con: 
- 'Cases_on_an_international_conveyance_Japan': irrelevante, para el estudio se toman encuenta las muertes en los **diferentes territorios nacionales**.

Datos del dataset5 relevantes para el estudio:

## Carga de Datos: Dataset 5
Carga del **Dataset5_Coronavirus_cases.txt**, cuyo continido son los **casos de Corona virus**, formato de archivo tipo **.txt**. Tipo de saparadores entre los datos **"|"**.

In [20]:
data5 = pd.read_csv("Dataset5_Coronavirus_cases.txt", sep="|", index_col="countriesAndTerritories")
data5.columns

Index(['dateRep', 'day', 'month', 'year', 'cases', 'deaths', 'geoId',
       'countryterritoryCode', 'popData2019', 'continentExp',
       'Cumulative_number_for_14_days_of_COVID-19_cases_per_100000'],
      dtype='object')

In [21]:
data5 = data5[['day', 'month', 'year', 'cases', 'deaths', 'continentExp',
       'Cumulative_number_for_14_days_of_COVID-19_cases_per_100000']]

## Limpieza de Datos 

Datos incompletos, mal organizados, errores, etc. Hay que intentar identificarlos y tratarlos antes de comenzar el análisis, o mediante borrado, sustitución, etc.

En este caso, afortundamente no tenemos datos perdidos.

In [22]:
data5 = data5.drop("Cases_on_an_international_conveyance_Japan")
data5.isnull().values.any()

True

## Profiling del dataset 5

Hacemos profiling del quinto dataset para poder analizar mejor los datos contenidos en el dataset final una vez lo hemos limpiado.

In [ ]:
profile = ProfileReport(data5, title="Coronavirus cases", html={'style':{'full_width': True}})
profile.to_notebook_iframe()